In [ ]:
# GET POR Files Already Downloaded
import glob
import shutil

source = '/Users/kris/Downloads/FFIEC/'
destination = '/Users/kris/Library/CloudStorage/OneDrive-Personal/Documents/Visual Studio 2019/Repos/ffiec-xbrl/PubData/Src/RSSDs/'

for src in glob.glob(source + '*/* POR *'):
    shutil.copy(src, destination)

In [1]:
# Imports and Constants
import pandas as pd
import numpy as np

#dtypes = {'RSSD_ID':str,
#          'FDIC_Certificate_Number':str,
#          'OCC_Charter_Number':str,
#          'OTS_Docket_Number':str,
#          'Primary_ABA_Routing_Number':str,
#          'Financial_Institution_Name': str,
#          'Financial_Institution_Address': str,
#          'Financial_Institution_City': str,
#          'Financial_Institution_State': str,
#          'Financial_Institution_Zip_Code': str,
#          'Financial_Institution_Filing_Type': str,
#          'Last Date/Time Submission Updated On': 'str'}

dtypes = {'IDRSSD': np.dtype('str'),
          'FDIC Certificate Number': np.dtype('str'),
          'OCC Charter Number': np.dtype('str'),
          'OTS Docket Number': np.dtype('str'),
          'Primary ABA Routing Number': np.dtype('str'),
          'Financial Institution Name': np.dtype('str'),
          'Financial Institution Address': np.dtype('str'),
          'Financial Institution City': np.dtype('str'),
          'Financial Institution State': np.dtype('str'),
          'Financial Institution Zip Code': np.dtype('str'),
          'Financial Institution Filing Type': np.dtype('str'),
          'Last Date/Time Submission Updated On': np.dtype('str')}

filepath_in = 'PubData/Src/RSSDs/'
filepath_out = 'PubData/'
filename = 'FFIEC CDR Call Bulk POR 03312024.txt'

In [ ]:
# Compare Bank_Dim and RSSD_Dict
df_bank = pd.read_csv(filepath_out + 'Bank_Dim.csv', sep=',', index_col=False, quotechar='"')
df_rssd = pd.read_csv(filepath_out + 'RSSD_Dict.csv', sep=',', index_col=False, quotechar='"')

print(df_bank.shape, df_rssd.shape)


In [2]:
#Open File (Bank_dim from RSSD_Por)

#with open(filename, 'r') as file:
df = pd.read_csv(filepath_in + filename, sep='\t', index_col=False, quotechar='"', dtype = dtypes, parse_dates=['Last Date/Time Submission Updated On'])

In [3]:
#Clean RSSD_Por File
df = df.apply(lambda x: x.str.strip() if x.dtype.name == 'object' else x, axis=0)

In [4]:
#Clean RSSD_Por Format - Align with and Create RSSD_Dict.csv
df.rename(columns={'IDRSSD': 'RSSD_ID'}, inplace=True)
df.columns = df.columns.str.replace(' ', '_')

#Create RSSD_Dict file
#df.iloc[:25].to_csv(filepath_out + 'RSSD_Dict.csv', sep=',', quotechar='"', index= False,)
df.to_csv(filepath_out + 'RSSD_Dict.csv', sep=',', quotechar='"', index= False)

In [ ]:
#Confirm data
df[:5]

#df.Financial_Institution_Address
df.iloc[0,6]

In [5]:
from FDIC import ExEngine
from FDIC import constants as paths
from FDIC.ETL import ETL
from FDIC.RateLimiter import RateLimiter 

rate_limiter = RateLimiter(max_calls=2450, period_in_seconds=3660) #~2.4k every 60min
etl = ETL(paths.WSDL_path, rate_limiter)
#ExEngine.FillMaster('504713')
#etl.GenBankDim()
etl.DownloadCallReports(['XBRL'])

#etl.ParseXBRL()
#etl.GenBankMaster()
#etl.GenCallMaster()

Already downloaded: BANK OF HANCOCK COUNTY_37
Already downloaded: FIRST COMMUNITY BANK XENIAFLORA_242
Already downloaded: BROADSTREET BANK SSB_279
Already downloaded: BISON STATE BANK_354
Already downloaded: LOWRY STATE BANK_457
Already downloaded: BALLSTON SPA NATIONAL BANK_505
Already downloaded: RVR BANK_1155
Already downloaded: BANK OF GRANDIN_1351
Already downloaded: HILLTOP NATIONAL BANK_1454
Already downloaded: LIBERTY NATIONAL BANK IN PARIS THE_1669
Already downloaded: WASHITA STATE BANK_1856
Already downloaded: LAMAR NATIONAL BANK_2161
Already downloaded: ALBANY BANK  TRUST CO NATIONAL ASSOCIATION_2732
Already downloaded: GENOA COMMUNITY BANK_2750
Already downloaded: FIRST NATIONAL BANK OF GORDON THE_3252
Already downloaded: FARMBANK_3458
Already downloaded: BANK OF CLARENDON THE_3720
Already downloaded: MIDCOUNTRY BANK_3971
Already downloaded: KANZA BANK_4156
Already downloaded: FARMERS AND MERCHANTS BANK_4231
Already downloaded: FIRST CREDIT BANK_5069
Already downloaded: FIR

KeyboardInterrupt: 

In [6]:
call_report_path = '/Users/kris/data_sources/fdic/BulkReports/ACNB BANK_5210/ACNB BANK_5210_20010331.XBRL'
bank_path = '/Users/kris/data_sources/fdic/BulkReports/ACNB BANK_5210/'
folder_path = '/Users/kris/data_sources/fdic/BulkReports/'

#tot = etl.ParseXBRL(call_report_path)
etl.GenBankMaster()
#etl.GenCallMaster()

Generate Bank Master:
	/Users/kris/data_sources/fdic/BulkReports/JOHNSON BANK_58243/58243_master
	/Users/kris/data_sources/fdic/BulkReports/NORTHFIELD BANK_28013/28013_master
	/Users/kris/data_sources/fdic/BulkReports/BANK OF EASTERN OREGON_14865/14865_master
	/Users/kris/data_sources/fdic/BulkReports/JOHNSON COUNTY BANK_45430/45430_master
	/Users/kris/data_sources/fdic/BulkReports/FIRST NATIONAL BANK_42448/42448_master
	/Users/kris/data_sources/fdic/BulkReports/INTERCREDIT BANK NATIONAL ASSOCIATION_44433/44433_master
	/Users/kris/data_sources/fdic/BulkReports/WAYNE BANK_59316/59316_master
	/Users/kris/data_sources/fdic/BulkReports/VANTAGE BANK TEXAS_26765/26765_master
	/Users/kris/data_sources/fdic/BulkReports/FIRST NATIONAL BANK OF WAUCHULA_89135/89135_master
	/Users/kris/data_sources/fdic/BulkReports/GALION BUILDING AND LOAN BANK THE_24378/24378_master
	/Users/kris/data_sources/fdic/BulkReports/SECURITY FEDERAL SAVINGS BANK_32971/32971_master
	/Users/kris/data_sources/fdic/BulkRepor

In [7]:
etl.GenCallMaster()

Generate Call Master:
	/Users/kris/data_sources/fdic/BulkReports/MasterCall.csv
	345 records processed in file


In [8]:
import FDIC.constants as paths
import pandas as pd
#tot

df = pd.read_csv(paths.localPath + paths.filename_MasterCall)


In [9]:
df.head(1)

,ReportPeriodEndDate,RSSD_ID,Item_Name,MDRM_Item,Confidential,Value
0,2018-12-31,58243,TOTAL QUALIFYING CAPITAL ALLOWABLE UNDER THE R...,RCOA3792,No,647895000


In [10]:
#Number of RSSD's by Period, by RSSD_ID, by MDRM_Item
g_df = df[df['RSSD_ID']==28013].groupby(['MDRM_Item', 'RSSD_ID', 'ReportPeriodEndDate'])['MDRM_Item'].count()
g_df

MDRM_Item  RSSD_ID  ReportPeriodEndDate
RCOA2170   28013    2020-03-31             1
                    2020-06-30             1
                    2020-09-30             1
                    2020-12-31             1
                    2021-03-31             1
                                          ..
TEXTC014   28013    2001-12-31             1
TEXTC016   28013    2001-03-31             1
                    2001-06-30             1
                    2001-09-30             1
                    2001-12-31             1
Name: MDRM_Item, Length: 97707, dtype: int64

In [11]:
#Number of RSSD's by field by date
pvt = df[['MDRM_Item', 'ReportPeriodEndDate']].pivot_table(
    index=['MDRM_Item'],
    columns=['ReportPeriodEndDate'],
    values=['MDRM_Item'],
    aggfunc={'MDRM_Item': ['count']}
)
pvt.fillna(0, inplace=True)
pvt

MDRM_Item                                              \
                         count                                               
ReportPeriodEndDate 2001-03-31 2001-06-30 2001-09-30 2001-12-31 2002-03-31   
MDRM_Item                                                                    
RCFA3792                   0.0        0.0        0.0        0.0        0.0   
RCFA5310                   0.0        0.0        0.0        0.0        0.0   
RCFA5311                   0.0        0.0        0.0        0.0        0.0   
RCFA7204                   0.0        0.0        0.0        0.0        0.0   
RCFA7205                   0.0        0.0        0.0        0.0        0.0   
...                        ...        ...        ...        ...        ...   
TEXTG561                   0.0        0.0        0.0        0.0        0.0   
TEXTG571                   0.0        0.0        0.0        0.0        0.0   
TEXTG576                   0.0        0.0        0.0        0.0        0.0   
TEXTG581                   0.0        0.0        0.0        0.0        0.0   
TEXTG586                   0.0        0.0        0.0        0.0        0.0   

                                                                            \
                                                                             
ReportPeriodEndDate 2002-06-30 2002-09-30 2002-12-31 2003-03-31 2003-06-30   
MDRM_Item                                                                    
RCFA3792                   0.0        0.0        0.0        0.0        0.0   
RCFA5310                   0.0        0.0        0.0        0.0        0.0   
RCFA5311                   0.0        0.0        0.0        0.0        0.0   
RCFA7204                   0.0        0.0        0.0        0.0        0.0   
RCFA7205                   0.0        0.0        0.0        0.0        0.0   
...                        ...        ...        ...        ...        ...   
TEXTG561                   0.0        0.0        0.0        0.0        0.0   
TEXTG571                   0.0        0.0        0.0        0.0        0.0   
TEXTG576                   0.0        0.0        0.0        0.0        0.0   
TEXTG581                   0.0        0.0        0.0        0.0        0.0   
TEXTG586                   0.0        0.0        0.0        0.0        0.0   

                     ...                                              \
                     ...                                               
ReportPeriodEndDate  ... 2021-12-31 2022-03-31 2022-06-30 2022-09-30   
MDRM_Item            ...                                               
RCFA3792             ...        8.0        8.0        8.0        8.0   
RCFA5310             ...        8.0        8.0        8.0        8.0   
RCFA5311             ...        8.0        8.0        8.0        8.0   
RCFA7204             ...        8.0        8.0        8.0        8.0   
RCFA7205             ...        8.0        8.0        8.0        8.0   
...                  ...        ...        ...        ...        ...   
TEXTG561             ...        0.0        0.0        0.0        0.0   
TEXTG571             ...        0.0        1.0        1.0        0.0   
TEXTG576             ...        0.0        0.0        0.0        0.0   
TEXTG581             ...        0.0        0.0        0.0        0.0   
TEXTG586             ...        0.0        0.0        0.0        0.0   

                                                                            \
                                                                             
ReportPeriodEndDate 2022-12-31 2023-03-31 2023-06-30 2023-09-30 2023-12-31   
MDRM_Item                                                                    
RCFA3792                   8.0        8.0        8.0        8.0        8.0   
RCFA5310                   8.0        8.0        8.0        8.0        8.0   
RCFA5311                   8.0        8.0        8.0        8.0        8.0   
RCFA7204                   8.0        8.0        8.0

In [12]:
#Number of RSSD's by date
pvt = df[['ReportPeriodEndDate', 'RSSD_ID']].drop_duplicates().reset_index().pivot_table(
    index=['ReportPeriodEndDate'],
    values=['RSSD_ID'],
    aggfunc={'RSSD_ID': ['count']}
).sort_values(by='ReportPeriodEndDate', ascending=False)

pvt = pvt.set_axis([col for col in pvt.keys()], axis=1)
#pvt.reset_index(inplace=True)

pvt.fillna(0, inplace=True)
pvt

,"(RSSD_ID, count)"
ReportPeriodEndDate,
2024-03-31,346
2023-12-31,346
2023-09-30,346
2023-06-30,346
2023-03-31,346
...,...
2002-03-31,312
2001-12-31,312
2001-09-30,312
